# File to show plots for the evolution runs

In [ ]:
from plotting import create_plot
import os
from helpers import RESULTS_DIR
from collections import defaultdict

# Get experiment names
experiments = defaultdict(list)
for f in os.listdir(RESULTS_DIR):
    if not os.path.isdir(f'{RESULTS_DIR}/{f}'):
        continue
    experiments['_'.join(f.split('_')[1:])].append(f)

for name in experiments.keys():
    print(name)
    create_plot(name, figsize=(5,3))
